# Identifying Reliable Airlines From Flight Data

The goal of this project is to determine which domestic US airlines provided the most reliable service in 2015. Various evaluations will be performed on the dataset and an objective decision will be determined for airlines that repeatedly outperform their competitors in delays and cancellations.

Dateset Source: https://www.kaggle.com/usdot/flight-delays <BR>
Original Source: U.S. Department of Transportation's (DOT) Bureau of Transportation Statistics https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236
 
    

## I. A Glimpse at the Raw Data

#### Table 1: Flights

In [6]:
%%bigquery
SELECT 
*
FROM `ba775-team-1b.2015_Flights.flights`
LIMIT 5;

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,5,1,AA,328,N4XBAA,DEN,DFW,530,...,855,20,0,0,None,14,0,6,0,0
1,2015,1,5,1,UA,1757,N76528,IAH,DEN,540,...,737,23,0,0,None,4,0,19,0,0
2,2015,1,5,1,UA,1135,N39450,BOS,ORD,645,...,1027,103,0,0,None,2,0,0,101,0
3,2015,1,5,1,WN,376,N436WN,BWI,SAT,910,...,1230,20,0,0,None,20,0,0,0,0
4,2015,1,5,1,US,445,N191UW,CLT,DEN,935,...,1153,32,0,0,None,23,0,6,3,0


#### Table 2: Airlines

In [7]:
%%bigquery
SELECT 
*
FROM `ba775-team-1b.2015_Flights.airlines`
LIMIT 5;

,IATA_Code,Airline
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


#### Table 3: Airports

In [8]:
%%bigquery
SELECT
*
FROM `ba775-team-1b.2015_Flights.airports`
LIMIT 5;

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ADK,Adak Airport,Adak,AK,USA,51.87796,-176.64603
1,ADQ,Kodiak Airport,Kodiak,AK,USA,57.74997,-152.49386
2,AKN,King Salmon Airport,King Salmon,AK,USA,58.67680,-156.64922
3,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619
4,BET,Bethel Airport,Bethel,AK,USA,60.77978,-161.83800


## II. Data Pre-Processing

#### 1.) Prior to running queries on the data, we wanted to filter out some irrelevant columns in our 'flights' table and create a date column.

In [10]:
%%bigquery
SELECT *
FROM `ba775-team-1b.2015_Flights.flight_FILTERED`
LIMIT 5

,Date,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,TAXI_OUT,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015-01-05,US,1850,PHL,MCO,650,4,15,16,0,0,None,12,0,4,0,0
1,2015-01-05,EV,4187,EWR,BNA,929,44,13,50,0,0,None,6,0,28,16,0
2,2015-01-05,US,567,PHX,MCO,955,6,30,19,0,0,None,13,0,0,6,0
3,2015-01-05,US,2174,DCA,LGA,1000,2,67,34,0,0,None,32,0,2,0,0
4,2015-01-05,WN,467,LAX,HOU,1000,8,18,21,0,0,None,13,0,1,7,0


#### 2.) For our machine learning model, we filtered the above table even further to feed the model more relevant information.

* Scheduled departure column was converted from integer to time.
* Delay type columns were converted from minutes to "yes/no(1/0)" to prevent overfitting. Rather than showing how many minutes the delay type contributed, the model would only be informed if that delay type existed.
* Removed unusable data in departure time and origin airport (detailed explanation in Section V).

In [1]:
%%bigquery
SELECT *
FROM `ba775-team-1b.ML.filtered_dataset`
LIMIT 5

,Date,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_TIME,DEPARTURE_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015-01-05,US,1850,PHL,MCO,06:50:00,4,1,0,1,0,0
1,2015-01-05,EV,4187,EWR,BNA,09:29:00,44,1,0,1,1,0
2,2015-01-05,US,567,PHX,MCO,09:55:00,6,1,0,0,1,0
3,2015-01-05,US,2174,DCA,LGA,10:00:00,2,1,0,1,0,0
4,2015-01-05,WN,467,LAX,HOU,10:00:00,8,1,0,1,1,0


## III. The Evaluations

#### 1.) What domestic airlines are in the dataset we are working with?

* American Airlines
* Alaska Airlines
* JetBlue Airways
* Delta Airlines
* Atlantic Southeast Airlines
* Frontier Airlines
* Hawaiian Airlines
* American Eagle Airlines
* Spirit Airlines
* Skywest Airlines
* United Airlines
* US Airways
* Virgin America
* Southwest Airlines

In [11]:
%%bigquery
SELECT 
*
FROM `ba775-team-1b.2015_Flights.airlines`
ORDER BY IATA_Code

,IATA_Code,Airline
0,AA,American Airlines Inc.
1,AS,Alaska Airlines Inc.
2,B6,JetBlue Airways
3,DL,Delta Air Lines Inc.
4,EV,Atlantic Southeast Airlines
5,F9,Frontier Airlines Inc.
6,HA,Hawaiian Airlines Inc.
7,MQ,American Eagle Airlines Inc.
8,NK,Spirit Air Lines
9,OO,Skywest Airlines Inc.


#### 2.) How many flights did these 14 airlines operate in 2015?

* 5.8 million

In [9]:
%%bigquery
SELECT
COUNT(*) AS Total_Flight_Count
FROM `ba775-team-1b.2015_Flights.flight_FILTERED`

,Total_Flight_Count
0,5819079


#### 3.) How many of these 5.8 million flights did each airline operate?

In [2]:
%%bigquery
SELECT
    F.AIRLINE AS IATA_Code,
    A.Airline AS Airline_Name,
    COUNT(F.FLIGHT_NUMBER) AS Flight_Count
FROM `ba775-team-1b.2015_Flights.flight_FILTERED` AS F
INNER JOIN `ba775-team-1b.2015_Flights.airlines` AS A
ON A.IATA_Code = F.AIRLINE
GROUP BY 
    F.AIRLINE, 
    A.AIRLINE
ORDER BY Flight_Count DESC

,IATA_Code,Airline_Name,Flight_Count
0,WN,Southwest Airlines Co.,1261855
1,DL,Delta Air Lines Inc.,875881
2,AA,American Airlines Inc.,725984
3,OO,Skywest Airlines Inc.,588353
4,EV,Atlantic Southeast Airlines,571977
5,UA,United Air Lines Inc.,515723
6,MQ,American Eagle Airlines Inc.,294632
7,B6,JetBlue Airways,267048
8,US,US Airways Inc.,198715
9,AS,Alaska Airlines Inc.,172521


#### 4.) What is the average delay time (in minutes) for each airline?

*See table below*

In [6]:
%%bigquery 
SELECT DISTINCT 
    F.AIRLINE AS IATA_Code, 
    A.airline AS Airline_Name, 
    ROUND(AVG(F.DEPARTURE_DELAY),2) as Average_delay
FROM `ba775-team-1b.2015_Flights.flight_FILTERED` AS F
INNER JOIN `ba775-team-1b.2015_Flights.airlines` AS A
ON F.AIRLINE = A.IATA_Code
GROUP BY 
    F.AIRLINE, 
    A.airline
ORDER BY Average_delay DESC

,IATA_Code,Airline_Name,Average_delay
0,NK,Spirit Air Lines,15.94
1,UA,United Air Lines Inc.,14.44
2,F9,Frontier Airlines Inc.,13.35
3,B6,JetBlue Airways,11.51
4,WN,Southwest Airlines Co.,10.58
5,MQ,American Eagle Airlines Inc.,10.13
6,VX,Virgin America,9.02
7,AA,American Airlines Inc.,8.90
8,EV,Atlantic Southeast Airlines,8.72
9,OO,Skywest Airlines Inc.,7.80


#### 5.) Out of all flights, what percentage of them are delayed by late aircraft?

*See table below*

In [7]:
%%bigquery
SELECT
    y.airline AS IATA_code, 
    z.airline AS Airline_Name, 
    ROUND((x.count_of_delay/y.total_flights)*100, 2) AS Delay_by_Aircraft_Percentage
FROM 
    (SELECT
        airline,
        COUNT(airline) as total_flights
    FROM `ba775-team-1b.2015_Flights.flight_FILTERED`
    GROUP BY airline) as y
INNER JOIN 
    (SELECT 
        airline,
    count(late_aircraft_delay) as count_of_delay,
    FROM `ba775-team-1b.2015_Flights.flight_FILTERED`
    WHERE late_aircraft_delay != 0
    GROUP BY airline) AS x
ON y.airline = x.airline
INNER JOIN `ba775-team-1b.2015_Flights.airlines` AS z
ON z.IATA_code = x.airline
ORDER BY Delay_by_Aircraft_Percentage DESC

,IATA_code,Airline_Name,Delay_by_Aircraft_Percentage
0,WN,Southwest Airlines Co.,12.93
1,F9,Frontier Airlines Inc.,12.81
2,B6,JetBlue Airways,11.86
3,MQ,American Eagle Airlines Inc.,10.90
4,OO,Skywest Airlines Inc.,10.61
5,NK,Spirit Air Lines,9.99
6,UA,United Air Lines Inc.,9.67
7,EV,Atlantic Southeast Airlines,9.58
8,VX,Virgin America,8.72
9,AA,American Airlines Inc.,7.93


#### 6.) Varying reasons for flight cancellation:

* Weather
* Carrier
* National Air System
* Security

In [7]:
%%bigquery
SELECT
  CASE
    WHEN F.CANCELLATION_REASON = 'A' THEN 'Carrier'
    WHEN F.CANCELLATION_REASON = 'B' THEN 'Weather'
    WHEN F.CANCELLATION_REASON = 'C' THEN 'National Air System'
  ELSE
  'Security'
END
  AS Cancellation_Reason,
  SUM(F.CANCELLED) AS Cancellation_Total,
FROM
  `ba775-team-1b.2015_Flights.flight_FILTERED` AS F
INNER JOIN
  `ba775-team-1b.2015_Flights.airlines` AS A
ON
  F.AIRLINE = A.IATA_Code
WHERE
  CANCELLATION_REASON IS NOT NULL
GROUP BY
  F.CANCELLATION_REASON
ORDER BY
  Cancellation_Total DESC

,Cancellation_Reason,Cancellation_Total
0,Weather,48851
1,Carrier,25262
2,National Air System,15749
3,Security,22


#### 7.) Percentage of cancellations for all flights:

*See table below*

In [11]:
%%bigquery
SELECT
  DISTINCT (F.AIRLINE) AS IATA_Code,
  A.airline as Airline_Name,
  ROUND((SUM(F.CANCELLED)/COUNT(F.AIRLINE))*100, 2) AS Cancellation_Percentage
FROM
  `ba775-team-1b.2015_Flights.flight_FILTERED` AS F
INNER JOIN
  `ba775-team-1b.2015_Flights.airlines` AS A
ON
  F.AIRLINE = A.IATA_Code
GROUP BY
  F.AIRLINE,
  A.airline
ORDER BY
  Cancellation_Percentage DESC

,IATA_Code,Airline_Name,Cancellation_Percentage
0,MQ,American Eagle Airlines Inc.,5.10
1,EV,Atlantic Southeast Airlines,2.66
2,US,US Airways Inc.,2.05
3,NK,Spirit Air Lines,1.71
4,OO,Skywest Airlines Inc.,1.69
5,B6,JetBlue Airways,1.60
6,AA,American Airlines Inc.,1.50
7,WN,Southwest Airlines Co.,1.27
8,UA,United Air Lines Inc.,1.27
9,VX,Virgin America,0.86


#### 8.) To futher examine cancellations, what percentage of all flights are cancelled solely due to the airline?
   
*See table below*

In [9]:
%%bigquery
SELECT
    y.airline AS IATA_code, 
    z.airline AS Airline_Name, 
    ROUND((x.count_of_cancellation/y.total_flights)*100, 2) AS Cancellation_by_Airline_Percentage
FROM 
    (SELECT
        airline,
    COUNT(airline) as total_flights
    FROM `ba775-team-1b.2015_Flights.flight_FILTERED`
    GROUP BY airline) as y
INNER JOIN 
    (SELECT 
        airline,
        count(cancellation_reason) as count_of_cancellation,
    FROM `ba775-team-1b.2015_Flights.flight_FILTERED`
    WHERE cancellation_reason = 'A'
    GROUP BY airline) AS x
ON y.airline = x.airline
INNER JOIN `ba775-team-1b.2015_Flights.airlines` AS z
ON z.IATA_code = x.airline
ORDER BY Cancellation_by_Airline_Percentage DESC

,IATA_code,Airline_Name,Cancellation_by_Airline_Percentage
0,MQ,American Eagle Airlines Inc.,0.84
1,EV,Atlantic Southeast Airlines,0.63
2,NK,Spirit Air Lines,0.56
3,UA,United Air Lines Inc.,0.56
4,OO,Skywest Airlines Inc.,0.54
5,US,US Airways Inc.,0.51
6,WN,Southwest Airlines Co.,0.49
7,AA,American Airlines Inc.,0.40
8,F9,Frontier Airlines Inc.,0.34
9,B6,JetBlue Airways,0.33


#### 9.) What percentage of these airlines' flights were diverted?

*See table below*

In [4]:
%%bigquery
SELECT
  DISTINCT (F.AIRLINE) AS IATA_Code,
  A.airline AS Airline_Name,
  ROUND((SUM(F.diverted)/COUNT(F.AIRLINE))*100, 3) AS Diversion_Percentage
FROM
  `ba775-team-1b.2015_Flights.flight_FILTERED` AS F
INNER JOIN
  `ba775-team-1b.2015_Flights.airlines` AS A
ON
  F.AIRLINE = A.IATA_Code
GROUP BY
  F.AIRLINE,
  A.airline
ORDER BY
  Diversion_Percentage DESC

,IATA_Code,Airline_Name,Diversion_Percentage
0,EV,Atlantic Southeast Airlines,0.349
1,AA,American Airlines Inc.,0.293
2,MQ,American Eagle Airlines Inc.,0.277
3,B6,JetBlue Airways,0.273
4,WN,Southwest Airlines Co.,0.270
5,UA,United Air Lines Inc.,0.269
6,OO,Skywest Airlines Inc.,0.268
7,AS,Alaska Airlines Inc.,0.239
8,US,US Airways Inc.,0.214
9,DL,Delta Air Lines Inc.,0.203


#### 10.) Selecting the top 3 and bottom 3 airlines

For each query where we compared percentages and average delay times, their tables were saved. Next, the following query was run to assign the airline's rank as points. For example: if the airline was in last place for lowest percentage of diversions, then it received 14 points. If it was in first place for highest percentage of diversion, then it received 1 point.

The points were tallied, and a top 3 and bottom 3 were chosen based on how many points the airline had.

Top 3 airlines (in order):
* Hawaiian Airlines
* Alaska Airlines
* Delta Airlines

Bottom 3 airlines (in order):
* United Airlines
* Atlantic Southeast Airlines
* American Eagle Airlines


In [12]:
%%bigquery
SELECT
*,
(Delay_Score + Delay_Percentage_Score + Cancellation_Score + Airline_Cancellation_Score + Diverted_Flights_Score) AS Total_Score
FROM 
(
SELECT
A.Airline_Name,
RANK() OVER(ORDER BY average_delay desc) as Delay_score,
RANK() OVER(ORDER BY Delay_by_Aircraft_Percentage desc) as Delay_Percentage_Score,
RANK() OVER(ORDER BY Cancellation_Percentage desc) as Cancellation_Score,
RANK() OVER(ORDER BY cancellation_by_Airline_percentage desc) as Airline_Cancellation_Score,
RANK() OVER(ORDER BY Diversion_Percentage desc) as Diverted_Flights_Score,

FROM `ba775-team-1b.final_presentation_tables.rank_average_delay` as A
INNER JOIN `ba775-team-1b.final_presentation_tables.rank_cancellation_airline` as B
ON A.iata_code = B.iata_code
INNER JOIN `ba775-team-1b.final_presentation_tables.rank_cancellation_percentage` as C
ON A.iata_code = C.iata_code
INNER JOIN `ba775-team-1b.final_presentation_tables.rank_delay_by_late_aircraft` as D
ON A.iata_code = D.iata_code
INNER JOIN `ba775-team-1b.final_presentation_tables.rank_percentage_diverted` AS E
ON A.iata_code = E.iata_code
)

ORDER BY Total_Score DESC

,Airline_Name,Delay_score,Delay_Percentage_Score,Cancellation_Score,Airline_Cancellation_Score,Diverted_Flights_Score,Total_Score
0,Hawaiian Airlines Inc.,14,12,14,12,14,66
1,Alaska Airlines Inc.,13,14,13,13,8,61
2,Delta Air Lines Inc.,11,13,12,14,10,60
3,Virgin America,7,9,10,11,11,48
4,US Airways Inc.,12,11,3,6,9,41
5,Frontier Airlines Inc.,3,2,11,9,12,37
6,American Airlines Inc.,8,10,7,8,2,35
7,Skywest Airlines Inc.,10,5,5,5,7,32
8,JetBlue Airways,4,3,6,10,4,27
9,Spirit Air Lines,1,6,4,3,13,27


## IV. Can we predict delay times?

To further build on our project, we wanted to run a machine learning model on our data to see how accurately we could predict delay times.

We created a simple linear regression model that took the following input:

* Departure delay (above 0 min and below 60 min)
* Day of week
* Hour of day
* Departure time
* Airline
* Origin airport
* Air system delay (yes = 1, no = 0)
* Security delay (yes = 1, no = 0)
* Airline delay (yes = 1, no = 0)
* Late aircraft delay (yes = 1, no = 0)
* Weather delay (yes = 1, no = 0)

#### 1.) This model below trained 80% of our data:

In [ ]:
CREATE or REPLACE MODEL ML.entire_dataset
OPTIONS (model_type = 'linear_reg', labels = ['departure_delay']) AS

SELECT 
departure_delay, 
EXTRACT(DAYOFWEEK from date) AS Day_of_week, 
departure_time, 
airline, 
origin_airport, 
destination_airport, 
air_system_delay, 
security_delay, 
airline_delay, 
late_aircraft_delay, 
weather_delay

FROM  `ba775-team-1b.ML.filtered_dataset`

WHERE
departure_delay IS NOT NULL AND
departure_delay > 0 AND 
departure_delay < 60 AND
MOD(ABS(FARM_FINGERPRINT(CAST(date AS STRING))),10) <= 7

#### 2.) When evaluated, we got a root mean squared error of 10.75. A glimpse into the 20% of data that was evaluated:

In [25]:
%%bigquery
SELECT *
FROM `ba775-team-1b.ML.entire_dataset_eval` 
ORDER BY RAND()
LIMIT 10

,predicted_departure_delay,departure_delay,Day_of_week,departure_time,airline,origin_airport,destination_airport,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,36.399096,31,4,08:45:00,B6,BOS,CLT,0,0,1,1,0
1,1.380354,1,3,14:38:00,HA,OGG,LIH,1,1,1,1,1
2,40.591888,44,4,22:55:00,DL,LAX,JFK,0,0,1,1,0
3,10.576394,1,6,09:34:00,OO,SBP,SFO,1,1,1,1,1
4,8.889813,3,2,18:30:00,WN,DAL,AUS,1,1,1,1,1
5,9.590538,9,5,16:35:00,AA,LAX,ORD,1,1,1,1,1
6,21.217879,12,2,11:15:00,WN,MDW,LAX,1,0,1,0,0
7,10.346293,5,7,14:59:00,B6,JFK,MCO,1,1,1,1,1
8,36.243421,53,4,16:45:00,VX,LAX,ORD,0,0,0,1,0
9,8.335066,9,2,22:10:00,WN,HOU,OAK,1,1,1,1,1


#### 3.) We created another model based off the first one, however this time we set the origin to just one airport and one airline: SFO, Southwest Airlines. When evaluating the model, we got an RMSE of 49.63.

In [26]:
%%bigquery
SELECT *
FROM `ba775-team-1b.ML.SFO_Southwest_eval` 
ORDER BY RAND()
LIMIT 10

,predicted_departure_delay,departure_delay,Day_of_week,departure_time,airline,origin_airport,destination_airport,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,11.916760,2,1,20:00:00,WN,SFO,LAS,1,1,1,1,1
1,-510.488424,42,1,08:50:00,WN,SFO,LAX,0,0,0,0,1
2,7.699439,2,5,14:00:00,WN,SFO,PHX,1,1,1,1,1
3,33.338020,37,3,15:30:00,WN,SFO,SAN,0,0,0,1,0
4,27.194926,23,2,19:05:00,WN,SFO,SNA,1,0,0,1,0
5,12.542407,13,1,16:35:00,WN,SFO,SAN,1,1,1,1,1
6,6.965096,6,3,11:55:00,WN,SFO,PHX,1,1,1,1,1
7,14.079350,21,3,20:10:00,WN,SFO,DEN,1,1,1,1,1
8,8.779242,4,2,18:20:00,WN,SFO,PHX,1,1,1,1,1
9,22.057056,8,3,15:00:00,WN,SFO,DEN,1,0,1,0,0


#### 4.) Lastly, we created a similar model but with the airline being only Southwest. When evaluating the model, we got an RMSE of 9.20.

In [33]:
%%bigquery
SELECT *
FROM `ba775-team-1b.ML.Southwest_eval` 
ORDER BY RAND()
LIMIT 10

,predicted_departure_delay,departure_delay,Day_of_week,departure_time,airline,origin_airport,destination_airport,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,9.377879,2,1,16:00:00,WN,LAS,MCO,1,1,1,1,1
1,31.360846,23,7,10:45:00,WN,BWI,MCO,0,0,1,0,0
2,10.366474,4,6,13:50:00,WN,LAX,SFO,1,1,1,1,1
3,7.106170,5,6,06:00:00,WN,BDL,MDW,1,1,1,1,1
4,37.477766,30,6,21:10:00,WN,PHX,PDX,0,0,0,1,0
5,7.607672,11,2,13:35:00,WN,STL,LAX,1,1,1,1,1
6,11.117003,10,5,20:40:00,WN,ATL,DTW,1,1,1,1,1
7,25.027664,22,4,12:30:00,WN,PIT,BWI,1,0,1,1,0
8,7.368314,2,5,06:30:00,WN,CLE,MDW,1,1,1,1,1
9,7.818530,1,1,14:10:00,WN,SJC,MDW,1,1,1,1,1


## V. Restrictions and Limitations

Ideally, we would have perfectly structured and informative data. However, that was not the case. Below are some setbacks we faced while working on this project.

#### 1.) Improper Airport Codes

We ecountered strange numeric numbers which could not be converted to a standard 3 character airport code. This resulted in us having to remove a portion of the data prior to training our machine learning model.

In [1]:
%%bigquery
SELECT origin_airport
FROM `ba775-team-1b.2015_Flights.flight_FILTERED`
WHERE length(origin_airport) > 3
LIMIT 5

,origin_airport
0,13930
1,14679
2,14747
3,13342
4,12892


#### 2.) Improper time objects

The times in the scheduled departure column were integer objects rather than time. As a result, we had to first pad 0's to the left of the number and then parse the column as a time object.

In [4]:
%%bigquery
SELECT scheduled_departure
FROM `ba775-team-1b.2015_Flights.flight_FILTERED`
LIMIT 5

,scheduled_departure
0,650
1,929
2,955
3,1000
4,1000


Solution:

In [13]:
%%bigquery
SELECT PARSE_TIME("%H%M", LPAD(CAST(scheduled_departure AS STRING), 4, '0')) as scheduled_time
FROM `ba775-team-1b.2015_Flights.flight_FILTERED`
LIMIT 5

,scheduled_time
0,06:50:00
1,09:29:00
2,09:55:00
3,10:00:00
4,10:00:00


#### 3.) Undeterminable Times

Also in the scheduled departure column were 2 character integers that were ambiguous. These had to be removed, which further elimated potential data we could have trained our model on. 

In [14]:
%%bigquery
SELECT scheduled_departure
FROM `ba775-team-1b.2015_Flights.flight_FILTERED`
WHERE length(cast(scheduled_departure AS STRING)) <= 2
LIMIT 5

,scheduled_departure
0,30
1,25
2,15
3,45
4,59


#### 4.) Bad predictions

We ran a simple regression model and got some fairly decent predictions. However, there were also some rather poor predictions as well. This could be attributed to a number of reasons:

1.) Lack of data. Having more relevant data information such as temperature, wind speed, or air-space congestion might have improved our model.

2.) Model is too simple. Perhaps if we had run a polynomial regression or something else beyond the scope of our course, we might have gotten more accurate predictions.

3.) Unforeseen problematic data. It is possible we may not have caught all the bad data like those listed above. Given that there are 5 million plus rows of data, this could be a possibility. 

In [17]:
%%bigquery
SELECT predicted_departure_delay, departure_delay
FROM `ba775-team-1b.ML.entire_dataset_eval` 
WHERE predicted_departure_delay < 0
ORDER BY predicted_departure_delay
LIMIT 5

,predicted_departure_delay,departure_delay
0,-2123.526203,15
1,-2.902266,9
2,-2.404074,22
3,-1.844094,20
4,-1.806976,4


## VI. Dashboard

Below is a link to our Tableau dashboard and screenshots of it to better visualize our project.

Link: https://public.tableau.com/views/Team1bDashboard/Team1bDashboard?:language=en&:display_count=y&:origin=viz_share_link

![](https://i.imgur.com/L4U7PbP.png)

![](https://i.imgur.com/v83vJFl.png)